# day 4-2

このノートブックの実行例は[こちら(HTML版)](https://github.com/haradatm/lecture-gssm2025/blob/main/notebooks-samples/day-4-2.html.zip)で確認できます

---

## 0. 準備

### 0.1 必要なパッケージのインストール

In [ ]:
%%capture
!sudo apt-get update
!sudo apt-get install -y automake autoconf perl libtool graphviz libgraphviz-dev
!pip install -U japanize_matplotlib pyvis pygraphviz mca

教材のダウンロード

In [ ]:
!git clone https://github.com/haradatm/lecture-gssm2025

In [ ]:
!ln -s lecture-gssm2025/notebooks/gssm_utils.py .

### 0.2 MeCab インストール (時間目安: 約3分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_mecab.sh >> install_mecab.log 2>&1
!tail -n 1 install_mecab.log

### 0.3 CaboCha インストール (時間目安: 約4分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_cabocha.sh >> install_cabocha.log 2>&1
!tail -n 1 install_cabocha.log

### 0.4 セッションの再起動

In [ ]:
import os
os.kill(os.getpid(), 9)

セッションの再起動後は,以下のセルから実行してください. **注意: これより前のセルを再度実行する必要はありません!**

---

### 0.5 辞書登録

追加したい形態素の情報を CSV ファイル(user_dic.csv)に追記する

In [ ]:
!echo '"泉質",-1,-1,1,名詞,一般,*,*,*,*,泉質,センシツ,センシツ,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"海鮮丼",-1,-1,1,名詞,一般,*,*,*,*,海鮮丼,カイセンドン,カイセンドン,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"スカイツリー",-1,-1,1,名詞,一般,*,*,*,*,スカイツリー,スカイツリー,スカイツリー,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"バスタオル",-1,-1,1,名詞,一般,*,*,*,*,バスタオル,バスタオル,バスタオル,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!cat ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

CSVファイル(user_dic.csv)をコンパイルして辞書(user.dic)を作成する

In [ ]:
!./tools/usr/local/libexec/mecab/mecab-dict-index \
-d ./tools/usr/local/lib/mecab/dic/ipadic \
-u ./tools/usr/local/lib/mecab/dic/ipadic/user.dic \
-f utf-8 -t utf-8 \
./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

### 0.6 確認

In [ ]:
import MeCab
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")

print(tagger.parse("この泉質は極上です"))
print(tagger.parse("この海鮮丼は美味しいです"))
print(tagger.parse("近くにスカイツリーがあります"))
print(tagger.parse("浴室にバスタオルがありません"))

In [ ]:
import CaboCha

cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
print(cp.parse("この泉質は極上です").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("この海鮮丼は美味しいです").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("近くにスカイツリーがあります").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("浴室にバスタオルがありません").toString(CaboCha.FORMAT_LATTICE))

---

## 1. テキスト分析 (2)

### 1.0 事前準備 (定義済み関数の読み込み)

#### 1.0.1 定義済み関数の読み込み

以下のセルを**修正せず**に実行してください

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random
import numpy as np

# 再現性のために乱数を固定する
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)

# 定義済み関数をインポートする
import warnings
warnings.simplefilter('ignore')

import gssm_utils

#### 1.0.1 データのダウンロード (前回ダウンロード済みのためスキップ)

以下のデータがダウンロード済みです

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2024-2025.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2024/1~2025 GW明け | 本講義の全体を通して使用する |

In [ ]:
# rakuten-1000-2024-2025.xlsx.zip をダウンロードする
FILE_ID = "1yKabZ7qJMRrIrP4Vtq-RrSZAqFsZriQS"
!gdown {FILE_ID}
!unzip -o rakuten-1000-2024-2025.xlsx.zip

#### 1.0.2 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

all_df = pd.read_excel("rakuten-1000-2024-2025.xlsx")
print(all_df.shape)
display(all_df.head())

#### 1.0.3 「文書-抽出語」表 を作成する (Top 1000語)

コメント列から単語を抽出する (単語を品詞「名詞」「形容詞」「未知語」で絞り込む)

In [ ]:
# 必要ライブラリのインポート
from collections import defaultdict
import MeCab

# mecab の初期化
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc --unk-feature 未知語")

# 単語頻度辞書の初期化
word_counts = defaultdict(lambda: 0)

# 抽出語情報リストの初期化
words = []

# 半角->全角変換マクロを定義する
ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# ストップワードを定義する
stopwords =  ["する", "それ", "なる", "ない", "そこ", "これ" ,"ある"]
stopwords += ["湯畑"]

# データ1行ごとのループ
for index, row in all_df.iterrows():

    # 半角->全角変換した後で, mecab で解析する
    node = tagger.parseToNode(row["コメント"].translate(HAN2ZEN))

    # 形態素ごとのループ
    while node:
        # 解析結果を要素ごとにバラす
        features = node.feature.split(',')

        # 品詞1 を取り出す
        pos1 = features[0]

        # 品詞2 を取り出す
        pos2 = features[1] if len(features) > 1 else ""

        # 原形 を取り出す
        base = features[6] if len(features) > 6 else None

        # 原型がストップワードに含まれない単語のみ抽出する
        if base not in stopwords:

            # 「名詞-一般」
            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「形容動詞」
            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}"
                postag = "形容動詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「形容詞」
            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「未知語」
            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

        # 次の形態素へ
        node = node.next

# DataFrme 型に整える
columns = [
    "文書ID",
    # "単語ID",
    "表層",
    "品詞",
    "カテゴリー",
    "エリア",
    "dict_key",
]
docs_df = pd.DataFrame(words, columns=columns)

# DataFrame を表示する
print(docs_df.shape)
display(docs_df.head())

抽出語の出現頻度をカウントする

In [ ]:
# 「文書-抽出語」 表から単語の出現回数をカウントする
word_list = []
for i, (k, v) in enumerate(sorted(word_counts.items(), key=lambda x:x[1], reverse=True)):
    word_list.append((i, k[0], v, k))

# DataFrame 型に整える
columns = [
    "単語ID",
    "表層",
    "出現頻度",
    "dict_key"
]

# DataFrame を表示する
word_counts_df = pd.DataFrame(word_list, columns=columns)
print(word_counts_df.shape)
display(word_counts_df.head(10))

「文書-抽出語」表を作成する (出現回数 Top 1000語)

In [ ]:
# 「単語出現回数」 表から出現回数Top 1000語のみ抽出する
word_counts_1000_df = word_counts_df[0:1000]

# 「文書-抽出語」 表も出現回数Top 150語のみに絞り込む
merged_df = pd.merge(docs_df, word_counts_1000_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_1000_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]

# 「カテゴリー,エリア」でクロス集計する
cross_1000_df = pd.crosstab(
    [
        docs_1000_df['カテゴリー'],
        docs_1000_df['エリア'],
        docs_1000_df['文書ID']
    ],
    docs_1000_df['単語ID'], margins=False
)
cross_1000_df.columns = word_counts_1000_df["表層"]

### 1.1 カテゴリーやエリアごとのユーザーの注目ポイントを押さえる

「文書-抽出語」表を {0,1} に変換する

In [ ]:
# 「文書-抽出語」 表を {0,1} に変換する
cross_1000_df[cross_1000_df > 0] = 1

# DataFrame を表示する
print(cross_1000_df.shape)
display(cross_1000_df)

#### 2.1.2 共起行列を作成する (外部変数-抽出語)

In [ ]:
# 「カテゴリー」のクロス集計と「エリア」のクロス集計を連結する
aggregate_df = pd.concat(
    [
        cross_1000_df.groupby(level='カテゴリー').sum(),
        cross_1000_df.groupby(level='エリア').sum()
    ]
)

# DataFrame を表示する
print(aggregate_df.shape)
display(aggregate_df)

#### 2.1.3 Jaccard 係数を求める (外部変数-抽出語)

In [ ]:
# 抽出語の出現回数を取得する
word_counts = cross_1000_df.sum(axis=0).values

# 属性(外部変数)出現数を取得する
attr_counts = np.hstack(
    [
        all_df.value_counts('カテゴリー').values,
        all_df.value_counts('エリア').values
    ]
)

# 共起行列の中身を Jaccard 係数に入れ替える
jaccard_attrs_df = gssm_utils.jaccard_attrs_coef(aggregate_df, attr_counts, word_counts, total=10000, conditional=False)

# DataFrame を表示する
print(jaccard_attrs_df.shape)
display(jaccard_attrs_df)

#### 2.1.4 特徴語ランキング

In [ ]:
# 「カテゴリ」や「エリア」ごとに Jaccard 係数のTop 10語を抽出する
df_list = []
for index, row in jaccard_attrs_df.iterrows():
    df_list.append(row.iloc[np.argsort(row.values)[::-1][:10]].reset_index())

# 「カテゴリ」や「エリア」ごとの Jaccard 係数のTop 10 を横方向に連結した DetaFrame を作成する
ranking_df = pd.DataFrame(pd.concat(df_list, axis=1))
ranking_df.columns = np.array([c for pair in [[x,f"jaccard"] for x in jaccard_attrs_df.index] for c in pair], dtype='object')

# DataFrame を表示する
display(ranking_df)

ファイルに出力する

In [ ]:
ranking_df.to_csv("practice-4.tsv", sep="\t")

#### 2.1.5 ワードクラウド

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# サブルーチン
def sort_and_plot(name, group):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # プロットする
    ax = fig.add_subplot(4, 3, i+1)
    gssm_utils.plot_wordcloud_ax(ax, " ".join(group_cross_df.columns))
    ax.set_title(name)


# プロットの準備
fig = plt.figure(figsize=(12, 9))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 2.1.6 共起ネットワーク図 (カテゴリ)

In [ ]:
# 抽出語の出現回数を取得する
word_counts = cross_1000_df.sum(axis=0).values

# 属性(外部変数)出現数を取得する
attr_counts = np.hstack(
    [
        all_df.value_counts('カテゴリー').values,
    ]
)

# カテゴリのみの共起行列(共起度数)を取得する
df = aggregate_df.loc[["A_レジャー","B_ビジネス"],:]

# 共起行列(共起度数)で共起ネットワーク図を作成する
gssm_utils.plot_attrs_network(df, attr_counts, word_counts, np.sort(df.values.reshape(-1))[::-1][60], width=8, height=8)

#### 2.1.7 共起ネットワーク図 (エリア)

In [ ]:
# 抽出語の出現回数を取得する
word_counts = cross_1000_df.sum(axis=0).values

# 属性(外部変数)出現数を取得する
attr_counts = np.hstack(
    [
        all_df.value_counts('エリア').values,
    ]
)

# カテゴリのみの共起行列(共起度数)を取得する
df = aggregate_df.iloc[2:,:]

# 共起行列((共起度数)で共起ネットワーク図を作成する
gssm_utils.plot_attrs_network(df, attr_counts, word_counts, np.sort(df.values.reshape(-1))[::-1][120], width=8, height=8, pyvis=True)

#### 2.1.8 トピックを抽出する (LDA)

In [ ]:
# 必要ライブラリのインポート
from sklearn.decomposition import LatentDirichletAllocation as LDA

# ライブラリ LDA によるトピック抽出
lda = LDA(max_iter=25, learning_method='batch', random_state=42, n_jobs=-1, n_components=6)
lda.fit(cross_1000_df.values)

# トピックごとに出現確率Top 20語を表示する
n_top_words = 20
feature_names = cross_1000_df.columns
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic # {topic_idx+1}:", end=" ")
    for i in topic.argsort()[:-n_top_words-1:-1]:
        print(feature_names[i], end=" ")
    print()

In [ ]:
# 必要ライブラリのインポート
from sklearn.decomposition import LatentDirichletAllocation as LDA

# ライブラリ LDA によるトピック抽出
lda = LDA(max_iter=25, learning_method='batch', random_state=42, n_jobs=-1, n_components=6)
lda.fit(cross_1000_df.values)

# トピックごとに出現確率Top 20語を表示する
n_top_words = 20
feature_names = cross_1000_df.columns
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic # {topic_idx+1}:", end=" ")
    for i in topic.argsort()[:-n_top_words-1:-1]:
        print(feature_names[i], end=" ")
    print()

ChatGPT を使ってトピックを説明する

プロンプトの例:
> 以下はトピックとトピックごとの高確率ワードです. これを読んで各トピックを簡潔に要約してください.
>
> Topic # 1	温泉 風呂 良い 美味しい 露天風呂 宿 部屋 最高 …

結果の例:
- Topic #1: 温泉・食事の満足度
- Topic #2: スタッフと家族向けのサービス
- Topic #3: 立地と利便性の高さ
- Topic #4: サービスや施設の不満点
- Topic #5: 部屋と設備の快適さ／不満
- Topic #6: フロント対応と設備の不足

In [ ]:
# トピックごとに出現確率Top 75語をプロットする
n_top_words = 75
gssm_utils.plot_topic_model(lda, feature_names, n_top_words)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# サブルーチン
def sort_and_plot(name, group):

    # 文書ごとのトピック比率を取得
    doc_topic_distributions = lda.transform(group.values)

    # 文書全体のトピック比率を計算（平均を取る）
    overall_topic_distribution = np.mean(doc_topic_distributions, axis=0)

    # プロットする
    ax = fig.add_subplot(4, 3, i+1)
    gssm_utils.plot_topic_distribution_ax(ax, overall_topic_distribution)
    ax.set_title(name)

# プロットの準備
fig = plt.figure(figsize=(12, 9))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

### 1.2 カテゴリーやエリアごとのユーザーの注目ポイントの評価の違いを見つける

「文書-抽出語」表を {0,1} に変換する

In [ ]:
# 「文書-抽出語」 表を {0,1} に変換する
cross_1000_df[cross_1000_df > 0] = 1

# DataFrame を表示する
print(cross_1000_df.shape)
display(cross_1000_df)

#### 1.2.2 共起行列を作成する (外部変数-抽出語)

In [ ]:
# 「カテゴリー」のクロス集計と「エリア」のクロス集計を連結する
aggregate_df = pd.concat(
    [
        cross_1000_df.groupby(level='カテゴリー').sum(),
        cross_1000_df.groupby(level='エリア').sum()
    ]
)

# DataFrame を表示する
print(aggregate_df.shape)
display(aggregate_df)

#### 1.2.3 Jaccard 係数を求める (外部変数-抽出語)

In [ ]:
# 抽出語の出現回数を取得する
word_counts = cross_1000_df.sum(axis=0).values

# 属性(外部変数)出現数を取得する
attr_counts = np.hstack(
    [
        all_df.value_counts('カテゴリー').values,
        all_df.value_counts('エリア').values
    ]
)

# 共起行列の中身を Jaccard 係数に入れ替える
jaccard_attrs_df = gssm_utils.jaccard_attrs_coef(aggregate_df, attr_counts, word_counts, total=10000, conditional=False)

# DataFrame を表示する
print(jaccard_attrs_df.shape)
display(jaccard_attrs_df)

#### 1.2.4 共起ネットワーク図

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# サブルーチン
def sort_and_plot(name, group):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 共起行列の中身を Jaccard 係数に入れ替える
    group_jaccard_df = gssm_utils.jaccard_coef(group_cooccur_df, group_cross_df)

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # プロットする
    ax = fig.add_subplot(4, 3, i+1)
    pyvis_plot = gssm_utils.plot_cooccur_network_ax(ax, group_jaccard_df, word_counts, np.sort(group_jaccard_df.values.reshape(-1))[::-1][120], pyvis=True, name=f"{name}.html")
    ax.set_title(name)
    display(pyvis_plot)

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 1.2.5 係り受け行列を作成する (抽出語-抽出語)

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix

# 共起行列を作成する
X = cross_1000_df.values
X = csc_matrix(X)
Xc = (X.T * X)
# 対角成分のみにする
Xc = np.triu(Xc.toarray())

# DataFrame 型に整える
cooccur_1000_df = pd.DataFrame(Xc, columns=cross_1000_df.columns, index=cross_1000_df.columns)

# チャンク(文節)から単語を取り出す
def get_words(tree, from_chunk, stopwords):

    # チャンク(文節)の開始位置を取得する
    beg = from_chunk.token_pos

    # チャンクの開始位置を取得する
    end = from_chunk.token_pos + from_chunk.token_size

    # 抽出語情報リストの初期化
    words = []

    # チャンク(文節)ごとのループ
    for i in range(beg, end):

        # チャンク中の形態素を取り出す
        token = tree.token(i)

        # 解析結果を要素ごとにバラす
        features = token.feature.split(',')

        # 品詞1 を取り出す
        pos1 = features[0]

        # 品詞2 を取り出す
        pos2 = features[1] if len(features) > 1 else ""

        # 原形 を取り出す
        base = features[6] if len(features) > 6 else None

        # 原型がストップワードに含まれない単語のみ抽出する
        if base not in stopwords:

            # 「名詞-一般」
            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)

                # 抽出語情報をリストに追加する
                words.append(key)

            # 「形容動詞」
            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}だ"
                postag = "形容動詞"
                key = (base, postag)

                # 抽出語情報をリストに追加する
                words.append(key)

            # 「形容詞」
            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)

                # 抽出語情報をリストに追加する
                words.append(key)

            # 「未知語」
            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)

                # 抽出語情報をリストに追加する
                words.append(key)

    # 抽出語情報をリストを返却する
    return words


# 必要ライブラリのインポート
import CaboCha

# cabocha の初期化
cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")

# 半角->全角変換マクロを定義する
ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# ストップワードを定義する
stopwords =  ["する", "それ", "なる", "ない", "そこ", "これ" ,"ある"]
stopwords += ["湯畑"]
stopwords += ['*']  # 原形に 「'*'」 が出力された場合に除去するため

# 係り受けペア辞書の初期化
pair_counts = defaultdict(lambda: 0)
pairs = []

# データ1行ごとのループ
for index, row in all_df.iterrows():

    # 半角->全角変換した後で, cabocha で解析する
    tree = cp.parse(row["コメント"].translate(HAN2ZEN))

    # 解析結果から空でないチャンク(文節)のリストを集める
    chunks = {}
    key = 0
    for i in range(tree.size()):
        tok = tree.token(i)
        if tok.chunk:
            chunks[key] = tok.chunk
            key += 1

    # 係り元と係り先の単語情報(原形と品詞)を集める
    from_words, to_words = [], []
    for from_chunk in chunks.values():
        # 係り先がなければスキップ
        if from_chunk.link < 0:
            continue

        # 係り先のチャンク(文節)を取得する
        to_chunk = chunks[from_chunk.link]

        # 係り元の単語情報(原形と品詞)を取得する
        from_words = get_words(tree, from_chunk, stopwords)

        # 係り先の単語情報(原形と品詞)を取得する
        to_words = get_words(tree, to_chunk, stopwords)

    # 係り受けペアと頻度を収集する
    for f in from_words:
        for t in to_words:
            key = (f[0], t[0])
            pair_counts[key] += 1


# 係り受け行列を初期化する (共起行列と同じ形)
Xd = np.zeros(cooccur_1000_df.shape)

# 係り受けペアを係り受け列に変換する
for (f,t), v in pair_counts.items():
    columns = list(cooccur_1000_df.columns)
    if f in columns and t in columns:
        i = columns.index(f)
        j = columns.index(t)
        Xd[i,j] = v

# DataFrme 型に整える
dep_1000_df = pd.DataFrame(Xd, columns=cooccur_1000_df.columns, index=cooccur_1000_df.columns)
print(dep_1000_df.shape)
display(dep_1000_df.head())

#### 1.2.6 係り受けネットワーク図

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# サブルーチン
def sort_and_plot(name, group, pos):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 係り受け行列(条件付き確率)を初期化する (共起行列と同じ形)
    Xd = np.zeros(group_cooccur_df.shape)

    # 係り受けペアを係り受け行列(条件付き確率)に変換する
    for (f,t), v in pair_counts.items():

        # 係り元と係り先の両方が列に含まれる
        columns = list(group_cooccur_df.columns)
        if f in columns and t in columns:
            i = columns.index(f)
            j = columns.index(t)

            # 条件付き確率(係り受け頻度/係り先出現回数)を求める
            Xd[i,j] = v / word_counts[i]

    # 係り受け行列(条件付き確率)を DataFrame 型に整える
    group_dependency_df = pd.DataFrame(Xd, columns=group_cross_df.columns, index=group_cross_df.columns)

    # プロットする
    ax = fig.add_subplot(4, 3, pos+1)
    pyvis_plot = gssm_utils.plot_dependency_network_ax(ax, group_dependency_df, word_counts, np.sort(group_dependency_df.values.reshape(-1))[::-1][120], pyvis=False, name=f"{name}.html")
    ax.set_title(name)
    # display(pyvis_plot)

    # 係り受けペアを条件付き確率で降順にソートし10件表示する
    Xc = group_dependency_df.values
    words = group_dependency_df.columns
    flat_indices = np.argsort(Xc.ravel())[::-1][:10]
    row_indices, col_indices = np.unravel_index(flat_indices, Xc.shape)
    for idx in range(len(flat_indices)):
        value = Xc[row_indices[idx], col_indices[idx]]
        print(f"[{name}] {words[row_indices[idx]]} - {words[col_indices[idx]]}: {value:.04f}")

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group, i)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group, i)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

### 1.3 高評価のエリアに倣って、低評価のエリアを改善するプランを提案する

#### 1.3.1 対照的な2エリアを選択する

In [ ]:
# コーディングルール
coding_pos = ["良い","美味しい","広い","多い","素晴らしい","嬉しい","気持ちよい","楽しい","近い","大きい","気持ち良い","温かい","早い","優しい","新しい","暖かい","快い","明るい","美しい","可愛い","満足"]
coding_neg = ["古い","無い","高い","悪い","小さい","狭い","少ない","寒い","遅い","熱い","欲しい","暑い","冷たい","遠い","臭い","暗い","うるさい","ない","無い","残念","改善","不満"]

In [ ]:
# DataFrame を初期化する
cross_1000_ps_df = cross_1000_df.copy()
cross_1000_ps_df['ポジ'] = 0
cross_1000_ps_df['ネガ'] = 0
cross_1000_ps_df['総合1-2'] = 0
cross_1000_ps_df['総合4-5'] = 0

# コーディングルールを適用する (ポジ・ネガ)
pos_index = docs_df['表層'].str.contains("|".join(coding_pos))
neg_index = docs_df['表層'].str.contains("|".join(coding_neg))
cross_1000_ps_df.loc[cross_1000_ps_df.index.get_level_values('文書ID').isin(docs_df.loc[pos_index, '文書ID']), 'ポジ'] = 1
cross_1000_ps_df.loc[cross_1000_ps_df.index.get_level_values('文書ID').isin(docs_df.loc[neg_index, '文書ID']), 'ネガ'] = 1

# コーディングルールを適用する (総合評価)
cross_1000_ps_df.loc[cross_1000_ps_df.index.get_level_values('文書ID').isin(all_df[all_df['総合'] <=2].index), '総合1-2'] = 1
cross_1000_ps_df.loc[cross_1000_ps_df.index.get_level_values('文書ID').isin(all_df[all_df['総合'] >=4].index), '総合4-5'] = 1
cross_1000_ps_df = cross_1000_ps_df[['ポジ','ネガ','総合1-2','総合4-5']]

# DataFrame を表示する
print(cross_1000_ps_df.shape)
display(cross_1000_ps_df)

In [ ]:
# 「カテゴリー」のクロス集計と「エリア」のクロス集計を連結する
aggregate_ps_df = pd.concat(
    [
        cross_1000_ps_df.groupby(level='カテゴリー').sum(),
        cross_1000_ps_df.groupby(level='エリア').sum()
    ]
)

# DataFrame を表示する
print(aggregate_ps_df.shape)
display(aggregate_ps_df)

In [ ]:
# 必要ライブラリのインポート
import mca

# ライブラリ mca による対応分析
ncols = aggregate_ps_df.shape[1]
mca_ben = mca.MCA(aggregate_ps_df, ncols=ncols, benzecri=False)

# 行方向および列方向の値を取り出す
row_coord = mca_ben.fs_r(N=2)
col_coord = mca_ben.fs_c(N=2)

# 固有値を求める
eigenvalues = mca_ben.L
total = np.sum(eigenvalues)
# 寄与率を求める
explained_inertia = 100 * eigenvalues / total

# 行方向および列方向のラベルを取得する
row_labels = aggregate_ps_df.index
col_labels = aggregate_ps_df.columns

# プロットする
gssm_utils.plot_coresp(row_coord, col_coord, row_labels, col_labels, explained_inertia)

#### 1.3.2 ポジティブ意見の共起ネットワーク図を作成する

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# コーディングルール
coding_or = coding_pos

# サブルーチン
def sort_and_plot(name, group):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # コーディングルールで絞り込む
    index = docs_df['表層'].str.contains("|".join(coding_or))
    group_cross_df = group_cross_df[group_cross_df.index.get_level_values('文書ID').isin(docs_df.loc[index, '文書ID'])]

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 共起行列の中身を Jaccard 係数に入れ替える
    group_jaccard_df = gssm_utils.jaccard_coef(group_cooccur_df, group_cross_df)

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # プロットする
    ax = fig.add_subplot(4, 3, i+1)
    pyvis_plot = gssm_utils.plot_cooccur_network_with_code_ax(ax, group_jaccard_df, word_counts, np.sort(group_jaccard_df.values.reshape(-1))[::-1][120], coding_or, pyvis=True, name=f"pos-{name}.html")
    ax.set_title(name)
    display(pyvis_plot)

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 1.3.3 ポジティブ意見の係り受けネットワーク図を作成する

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# コーディングルール
coding_or = coding_pos

# サブルーチン
def sort_and_plot(name, group, pos):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # コーディングルールで絞り込む
    index = docs_df['表層'].str.contains("|".join(coding_or))
    group_cross_df = group_cross_df[group_cross_df.index.get_level_values('文書ID').isin(docs_df.loc[index, '文書ID'])]

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 係り受け行列(条件付き確率)を初期化する (共起行列と同じ形)
    Xd = np.zeros(group_cooccur_df.shape)

    # 係り受けペアを係り受け行列(条件付き確率)に変換する
    for (f,t), v in pair_counts.items():

        # 係り元と係り先の両方が列に含まれる
        columns = list(group_cooccur_df.columns)
        if f in columns and t in columns:
            i = columns.index(f)
            j = columns.index(t)

            # 条件付き確率(係り受け頻度/係り先出現回数)を求める
            Xd[i,j] = v / word_counts[i]

    # 係り受け行列(条件付き確率)を DataFrame 型に整える
    group_dependency_df = pd.DataFrame(Xd, columns=group_cross_df.columns, index=group_cross_df.columns)

    # プロットする
    ax = fig.add_subplot(4, 3, pos+1)
    pyvis_plot = gssm_utils.plot_cooccur_network_with_code_ax(ax, group_dependency_df, word_counts, np.sort(group_dependency_df.values.reshape(-1))[::-1][120], coding_or, pyvis=False, name=f"pos-{name}.html")
    ax.set_title(name)
    # display(pyvis_plot)

    # 係り受けペアを条件付き確率で降順にソートし10件表示する
    Xc = group_dependency_df.values
    words = group_dependency_df.columns
    flat_indices = np.argsort(Xc.ravel())[::-1][:20]
    row_indices, col_indices = np.unravel_index(flat_indices, Xc.shape)
    for idx in range(len(flat_indices)):
        if words[row_indices[idx]] in coding_or or words[col_indices[idx]] in coding_or:
            value = Xc[row_indices[idx], col_indices[idx]]
            print(f"[{name}] {words[col_indices[idx]]} - {words[row_indices[idx]]}: {value:.04f}")

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group, i)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group, i)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 1.3.4 ネガティブ意見の共起ネットワーク図を作成する

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# コーディングルール
coding_or = coding_neg

# サブルーチン
def sort_and_plot(name, group):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # コーディングルールで絞り込む
    index = docs_df['表層'].str.contains("|".join(coding_or))
    group_cross_df = group_cross_df[group_cross_df.index.get_level_values('文書ID').isin(docs_df.loc[index, '文書ID'])]

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 共起行列の中身を Jaccard 係数に入れ替える
    group_jaccard_df = gssm_utils.jaccard_coef(group_cooccur_df, group_cross_df)

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # プロットする
    ax = fig.add_subplot(4, 3, i+1)
    pyvis_plot = gssm_utils.plot_cooccur_network_with_code_ax(ax, group_jaccard_df, word_counts, np.sort(group_jaccard_df.values.reshape(-1))[::-1][120], coding_or, pyvis=True, name=f"neg-{name}.html")
    ax.set_title(name)
    display(pyvis_plot)

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 1.3.5 ネガティブ意見の係り受けネットワーク図を作成する

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# コーディングルール
coding_or = coding_neg

# サブルーチン
def sort_and_plot(name, group, pos):

    # 「カテゴリー」ごとに Jaccard 係数でソートする
    sorted_columns = np.argsort(jaccard_attrs_df.loc[name].values)[::-1][:75]

    # Jaccard 係数Top 75語をソートして抽出する
    group_cross_df = group.iloc[:,sorted_columns]

    # 抽出語の出現回数を取得する
    word_counts = group_cross_df.sum(axis=0).values

    # 共起行列を作成する
    X = group_cross_df.values
    X = csc_matrix(X)
    Xc = (X.T * X)
    Xc = np.triu(Xc.toarray())

    # コーディングルールで絞り込む
    index = docs_df['表層'].str.contains("|".join(coding_or))
    group_cross_df = group_cross_df[group_cross_df.index.get_level_values('文書ID').isin(docs_df.loc[index, '文書ID'])]

    # 共起行列を DataFrame に整える
    group_cooccur_df = pd.DataFrame(Xc, columns=group_cross_df.columns, index=group_cross_df.columns)

    # 係り受け行列(条件付き確率)を初期化する (共起行列と同じ形)
    Xd = np.zeros(group_cooccur_df.shape)

    # 係り受けペアを係り受け行列(条件付き確率)に変換する
    for (f,t), v in pair_counts.items():

        # 係り元と係り先の両方が列に含まれる
        columns = list(group_cooccur_df.columns)
        if f in columns and t in columns:
            i = columns.index(f)
            j = columns.index(t)

            # 条件付き確率(係り受け頻度/係り先出現回数)を求める
            Xd[i,j] = v / word_counts[i]

    # 係り受け行列(条件付き確率)を DataFrame 型に整える
    group_dependency_df = pd.DataFrame(Xd, columns=group_cross_df.columns, index=group_cross_df.columns)

    # プロットする
    ax = fig.add_subplot(4, 3, pos+1)
    pyvis_plot = gssm_utils.plot_cooccur_network_with_code_ax(ax, group_dependency_df, word_counts, np.sort(group_dependency_df.values.reshape(-1))[::-1][120], coding_or, pyvis=False, name=f"pos-{name}.html")
    ax.set_title(name)
    # display(pyvis_plot)

    # 係り受けペアを条件付き確率で降順にソートし10件表示する
    Xc = group_dependency_df.values
    words = group_dependency_df.columns
    flat_indices = np.argsort(Xc.ravel())[::-1][:10]
    row_indices, col_indices = np.unravel_index(flat_indices, Xc.shape)
    for idx in range(len(flat_indices)):
        if words[row_indices[idx]] in coding_or or words[col_indices[idx]] in coding_or:
            value = Xc[row_indices[idx], col_indices[idx]]
            print(f"[{name}] {words[col_indices[idx]]} - {words[row_indices[idx]]}: {value:.04f}")

# プロットの準備
# fig = plt.figure(figsize=(20, 28))
fig = plt.figure(figsize=(28, 28))

i = 0
# カテゴリごとのループ
for name, group in cross_1000_df.groupby(level='カテゴリー'):
    # サブルーチンを呼ぶ
    sort_and_plot(name, group, i)
    i += 1

    # エリアごとのループ
    for sub_name, sub_group in group.groupby(level='エリア'):
        # サブルーチンを呼ぶ
        sort_and_plot(sub_name, sub_group, i)
        i += 1

# プロットの仕上げ
plt.tight_layout()
plt.show()

#### 1.3.4 本文の参照 (エリアで絞る)

「登別」と「道後」で「すばらしい」という単語が含まれている口コミを表示する

In [ ]:
# 検索条件
search_index = \
    all_df['エリア'].isin(['01_登別', '04_道後']) & \
    (all_df['コメント'].str.contains('素晴らしい') | all_df['コメント'].str.contains('すばらしい'))

# 検索する
result_df = all_df[search_index]

# DataFrame を表示する
print(result_df.shape)
display(result_df.head())

# CSV に保存する
result_df.to_csv("output-1.csv", header=True)

「東京」と「福岡」で「うるさい」という単語が含まれている口コミを表示する

In [ ]:
# 検索条件
search_index = \
    all_df['エリア'].isin(['08_東京', '10_福岡']) & \
    all_df['コメント'].str.contains('うるさい')

# 検索する
result_df = all_df[search_index]

# DataFrame を表示する
print(result_df.shape)
display(result_df.head())

# CSV に保存する
result_df.to_csv("output-2.csv", header=True)